In [2]:
import json
import requests
from src import utils
import time
import os

In [3]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [4]:
api = utils.read_json_file(api_path)

In [5]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [6]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [7]:
os.makedirs(output_path, exist_ok=True)
output_file_name = 'result-' + input_file.split('/')[-1]
output_file = 'zero-shot-cot-result-ko-gsm8k-test.jsonl'
print(f'output_file : {output_file}')

output_file : zero-shot-cot-result-ko-gsm8k-test.jsonl


In [8]:
output_file = output_path + '/' + 'test.jsonl'
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/test.jsonl


In [9]:
jsonl_writer = utils.JSONLWriter(output_file)

In [10]:
data = utils.read_jsonlines(input_file)

In [11]:
len(data)

1319

In [12]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [13]:
model = model_api(api['url'], api['headers'])

In [14]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [15]:
# 차근차근 -> 단답 비율 높음
### 차근차근 생각해보자 0
### 차근 차근 풀어보면 0
### 단계적으로 생각해보면 1(reasoning/caculation 둘다 오답)
### 단계적으로 생각하면
### 단계별로 생각해보면 

# 단계벌 -> 단계별 접근
# 단계별로 질문에 대답해보면 0
# 단계별로 질문에 접근해보면 0

# 각 단계가 숫자로 구분되어짐 (음식의 경우, 조리방법)
# 단계별로 질문을 분석하면 0
# 질문을 단계적으로 풀어보면, 0



 

def get_gsm8k_zs_cot(que):
    prompt = f"질문: {que} 정답: 질문을 단계적으로 풀어보면, "
    return prompt

In [16]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [17]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [18]:
cnt = 0
split_cnt = 0
for ins in data:
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    kor_ans = ins['kor_answer']
    prompt = get_gsm8k_zs_cot(kor_que)
    ans_s = ins['answer_s']
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print(f'ans_s : ')
    print(ans_s)
    print('----')
    
    ins['kor_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 20:
        break
    else:
        cnt += 1

/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 정답 : 질문을 단계적으로 풀어보면, 
res_context : 
질문: 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 단계: 1. 섬유의 한 쌍을 나열합니다. 2. 파란색 섬유 2덩이와 흰색 섬유 1덩이가 있습니다. 3. 총 섬유 3덩이가 필요합니다. 4. 로브의 길이를 계산합니다. 5. 로브의 길이 = 섬유의 한 쌍 + 흰색 섬유의 한 쌍 + 파란색 섬유의 한 쌍 + 흰색 섬유의 한 쌍 = 5 + 6 = 8 6. 로브의 총 개수를 계산합니다. 7. 로브의 총 개수 = 섬유의 한 쌍 + 흰색 섬유의 한 쌍 + 파란색 섬유의 한 쌍 + 흰색 섬유의 한 쌍 + 파란색 섬유의 한 쌍 + 흰색 섬유의 한 쌍 = 8 + 9 = 11 10. 로브의 총 개수를 출력합니다. 11. 로브의 총 개수 = 로브의 총 개수 = 2 + 3 + 4 + 5 + 6 + 7 + 8 = 12 따라서, 한 벌의 로브를 만드는 데 파란색 섬유 2덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 섬유의 총 개수는 12입니다.
ans_s : 
3
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 조쉬는 집을 뒤집어 판매해보기로 결정합니다. 그는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 그가 얼마의 이익을 얻었습니까? 정답 : 질문을 단계적으로 풀어보면, 
res_context : 
조쉬는 집을 팔기로 결심하고 가격을 $80,000로 설정합니다. 그런 다음 수리 비용을 $50,000로 조정합니다. 조쉬는 $80,000 - $50,000 = $45,000을 벌게 됩니다. 이익은 $45,000 - $45,000 = $12,000입니다. 따라서 조쉬는 $12,000의 이익을 얻었습니다.
ans_s : 
70000
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
질문 : 한 달 동안 새로운 프로그램은 60번 다운로드되었습니다. 두 번째 달의 다운로드 횟수는 첫 번째 달의 다운로드 횟수의 3배였지만, 세 번째 달에는 30% 감소했습니다. 이 프로그램은 총 3개월 동안 얼마나 많이 다운로드되었습니까? 정답 : 질문을 단계적으로 풀어보면, 
res_context : 
1. 첫 번째 달과 두 번째 달의 다운로드 횟수를 각각 x, y로 표시합니다. 2. 세 번째 달의 다운로드 횟수를 z, k로 표시합니다. 3. 첫 번째 달과 세 번째 달의 다운로드 횟수를 더합니다. 4. (x + y) / 3 = 6 5. 6번째 달의 다운로드 횟수는 첫 번째 달의 다운로드 횟수의 3배이므로 3배입니다. 6. 세 번째 달의 다운로드 횟수는 첫 번째 달의 다운로드 횟수의 30%이므로 30%입니다. 7. 세 번째 달의 총 다운로드 횟수는 6 + (x + y) / 3 = 18 8. 총 다운로드 횟수는 18 + (x + y) / 3 = 24 9. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 10. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 11. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 12. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 13. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 14. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 15. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 16. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 17. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 18. 총 다운로드 횟수는 24 + (x + y) / 3 = 24 19. 총 다운로드 횟수는 24 + (x + y) / 3
ans_s : 
366
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 툴라는 빵집에 가서 다양한 종류의 과자를 샀습니다. 그녀는 3다스의 도넛을 1다스당 68달러에, 2다스의 미니 컵케이크를 1다스당 80달러에, 그리고 6다스의 미니 치즈케이크를 1다스당 55달러에 샀습니다. 총 비용은 얼마인가요? 정답 : 질문을 단계적으로 풀어보면, 
res_context : 
1. 3다스의 도넛을 산 총 비용은 68달러입니다. 2. 2다스의 미니 컵케이크를 산 총 비용은 80달러입니다. 3. 6다스의 미니 치즈케이크를 산 총 비용은 55달러입니다. 따라서 총 비용은 68달러 + 80달러 + 55달러 = 110달러입니다.
ans_s : 
694
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 카를로스는 레몬 나무를 심고 있습니다. 나무를 심는 데 90달러가 듭니다. 매년 7개의 레몬이 열리며, 그것을 1.5달러에 팔 수 있습니다. 나무를 물고 먹이는 데 매년 3달러가 듭니다. 레몬 나무에서 돈을 벌기 시작하는 데 몇 년이 걸릴까요? 정답 : 질문을 단계적으로 풀어보면, 
res_context : 
1. 나무를 사는 데 90달러를 씁니다. 2. 매년 7개의 레몬을 판습니다. 3. 1.5달러에 팔 수 있습니다. 4. 나무를 물어 먹는 데 매년 3달러가 듭니다. 5. 레몬 나무에서 돈을 버는 데 걸리는 시간은 다음과 같습니다: 6년 (2003년부터)
ans_s : 
13
----
